# Check the discrepancy of pseudo label and real label 

If the samples which the discrepancy of pseudo label and real label is large have good alignment with discriminator output, it will be good

In [1]:
import numpy as np
import json
from collections import defaultdict
from tqdm import tqdm
from copy import deepcopy


In [2]:
with open("initial_unseen_pseudo_label_old.json") as f:
    pseudo_label_data = json.load(f)
with open("/mnt/home/datasets/manual/coco_2017/annotations/instances_train2017.json") as f:
    real_label_data = json.load(f)

In [3]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) for two bounding boxes.
    box1 and box2 are in the format [x_min, y_min, width, height].
    """
    x1_min, y1_min, w1, h1 = box1
    x2_min, y2_min, w2, h2 = box2

    x1_max, y1_max = x1_min + w1, y1_min + h1
    x2_max, y2_max = x2_min + w2, y2_min + h2

    # Calculate intersection
    xi_min = max(x1_min, x2_min)
    yi_min = max(y1_min, y2_min)
    xi_max = min(x1_max, x2_max)
    yi_max = min(y1_max, y2_max)

    if xi_max < xi_min or yi_max < yi_min:
        return 0.0  # No intersection

    intersection = (xi_max - xi_min) * (yi_max - yi_min)

    # Calculate union
    area1 = w1 * h1
    area2 = w2 * h2
    union = area1 + area2 - intersection

    return intersection / union

In [26]:
# def evaluate_predictions(gt_annotations, pred_annotations, class_id, iou_threshold=0.5):
def evaluate_predictions(gt_annotations, pred_annotations, class_id_set, iou_threshold=0.5):
    """
    Evaluate predictions using Average Precision (AP) for a single class.
    gt_annotations: List of ground truth annotations.
    pred_annotations: List of predictions sorted by confidence score (descending).
    class_id: The class ID to evaluate.
    """
    # Filter annotations for the target class
    
    gt_annotations = [ann for ann in gt_annotations["annotations"]]
    pred_annotations = [pred for pred in pred_annotations["annotations"]]
    
    gt_by_image = defaultdict(list)
    pred_by_image = defaultdict(list)

    # Organize ground truth and predictions by image_id
    for ann in gt_annotations:
        # gt_by_image[ann["image_id"]].append(ann["bbox"])
        gt_by_image[ann["image_id"]].append((ann))

    for pred in pred_annotations:
        # pred_by_image[pred["image_id"]].append((pred["bbox"]))
        pred_by_image[pred["image_id"]].append((pred))

    result_dict = {}

    # for pred_bbox in preds:
    # for preds in pred_by_image_class_id:
    for pred_key in pred_by_image:
        best_iou = 0.0
        for class_id in class_id_set:
            gt_by_image_class_id_list = [ann_elem for ann_elem in gt_by_image[pred_key] if ann_elem["category_id"] == class_id] # object
            # pred_by_image_class_id_list = [deepcopy(ann_elem) for ann_elem in pred_by_image[pred_key] if ann_elem["category_id"] == class_id]
            pred_by_image_class_id_list = [ann_elem for ann_elem in pred_by_image[pred_key] if ann_elem["category_id"] == class_id]

            # for pred_bbox in pred_by_image[preds_key]:
            # for pred_bbox in preds:
            for pred in pred_by_image_class_id_list:
                pred_bbox = pred["bbox"]
                # for idx, gt_bbox in enumerate(gt_by_image[preds_key]):
                for gt in gt_by_image_class_id_list:
                    gt_bbox = gt["bbox"]
                    iou = calculate_iou(pred_bbox, gt_bbox)
                    if iou > best_iou:
                        best_iou = iou

        result_dict[pred_key] = best_iou

    return result_dict

In [27]:
def sort_predictions_by_ap(gt_annotations, pred_annotations, class_ids, iou_threshold=0.5):
    """
    Sort predictions by AP for each class.
    """
    ap_scores = []

    # for class_id in class_ids:
    ap = evaluate_predictions(gt_annotations, pred_annotations, class_ids, iou_threshold)
    # ap_scores.append({"class_id": class_id, "ap": ap})


    # Sort by AP in ascending order
    # sorted_ap_scores = sorted(ap_scores, key=lambda x: x["ap"])
    sorted_iou_scores = sorted(ap.items(), key=lambda x: x[1])
    return sorted_iou_scores


In [28]:
# Get unique class IDs from the ground truth
class_ids = set()
real_label_data_annotations = tqdm(real_label_data["annotations"])
# for ann in real_label_data["annotations"]:
for ann in real_label_data_annotations:
    class_ids.add(ann["category_id"])
    if len(class_ids) == 80:
        break

# Sort predictions by AP for each class
sorted_iou_scores = sort_predictions_by_ap(real_label_data, pseudo_label_data, class_ids)
print(sorted_iou_scores)

  0%|          | 0/860001 [00:00<?, ?it/s]

  8%|▊         | 71235/860001 [00:00<00:00, 1897674.41it/s]


[(133, 0.0), (2349, 0.0), (2645, 0.0), (3518, 0.0), (3737, 0.0), (3982, 0.0), (4309, 0.0), (4462, 0.0), (6211, 0.0), (6497, 0.0), (9520, 0.0), (11508, 0.0), (11527, 0.0), (11818, 0.0), (11825, 0.0), (12015, 0.0), (12711, 0.0), (12824, 0.0), (13035, 0.0), (14533, 0.0), (16040, 0.0), (17205, 0.0), (17859, 0.0), (18476, 0.0), (19783, 0.0), (20527, 0.0), (20692, 0.0), (20997, 0.0), (21046, 0.0), (21795, 0.0), (21964, 0.0), (22880, 0.0), (22983, 0.0), (23274, 0.0), (24091, 0.0), (24430, 0.0), (24664, 0.0), (25378, 0.0), (27075, 0.0), (27463, 0.0), (27793, 0.0), (28806, 0.0), (29085, 0.0), (29322, 0.0), (30276, 0.0), (30407, 0.0), (30646, 0.0), (31280, 0.0), (31288, 0.0), (31392, 0.0), (31946, 0.0), (32054, 0.0), (32416, 0.0), (32677, 0.0), (32909, 0.0), (33182, 0.0), (34389, 0.0), (34608, 0.0), (35328, 0.0), (35701, 0.0), (36349, 0.0), (36745, 0.0), (37502, 0.0), (39150, 0.0), (39685, 0.0), (39993, 0.0), (40466, 0.0), (40580, 0.0), (40596, 0.0), (40962, 0.0), (40966, 0.0), (41074, 0.0), (41

In [30]:
zero_iou_list = []
for sorted_iou_score in sorted_iou_scores:
    if sorted_iou_score[1] == 0.0:
        zero_iou_list.append(sorted_iou_score)
    else:
        continue

print(len(zero_iou_list))

969


In [31]:
len(sorted_iou_scores)


11161

In [ ]:
file_name_list = []
for zero_iou in zero_iou_list:
    image_id, iou_score = zero_iou
    for image in real_label_data["images"]:
        if image["id"] == image_id:
            # print(image["file_name"])
            file_name_list.append(image["file_name"])

In [ ]:
with open("zero_iou_file_name_list.txt", "w") as f:
    for file_name in file_name_list:
        f.write(file_name + "\n")

In [34]:
real_label_data["images"]

[{'license': 3,
  'file_name': '000000391895.jpg',
  'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
  'height': 360,
  'width': 640,
  'date_captured': '2013-11-14 11:18:45',
  'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
  'id': 391895},
 {'license': 4,
  'file_name': '000000522418.jpg',
  'coco_url': 'http://images.cocodataset.org/train2017/000000522418.jpg',
  'height': 480,
  'width': 640,
  'date_captured': '2013-11-14 11:38:44',
  'flickr_url': 'http://farm1.staticflickr.com/1/127244861_ab0c0381e7_z.jpg',
  'id': 522418},
 {'license': 3,
  'file_name': '000000184613.jpg',
  'coco_url': 'http://images.cocodataset.org/train2017/000000184613.jpg',
  'height': 336,
  'width': 500,
  'date_captured': '2013-11-14 12:36:29',
  'flickr_url': 'http://farm3.staticflickr.com/2169/2118578392_1193aa04a0_z.jpg',
  'id': 184613},
 {'license': 3,
  'file_name': '000000318219.jpg',
  'coco_url': 'http://images.cocodataset.org/train2017/0